### Configure your ArcGIS Online or Enterprise cnonnections, and specify a group from your src_gis to be copied.

In [1]:
import arcgis
src_gis_url = 'https://nyeveier.maps.arcgis.com/'     #https://grid3.maps.arcgis.com - can also use 'home' if you are running the notebook from the org you want to use as the src. 
src_group_id = 'groupid'
src_gis_username = 'adminbruker' # Your username for the source organization
src_gis_password = 'passord' # provide a password for that user, if None is left, you will be prompted to provide the password.

target_gis_url = 'https://nyeveieras.maps.arcgis.com/'    #https://myneworg.maps.arcgis.coNm
target_gis_username = 'adminbruker' # Username for Target Org
target_gis_password = 'passord' # Password for Target Org
target_folder_name = 'nymappe' #Name for the Folder in the target user's content that you want to copy content into.

src_gis = arcgis.gis.GIS(src_gis_url, src_gis_username, src_gis_password)
#src_gis = arcgis.gis.GIS(src_gis_url,client_id='f5nBiaq7vzWKw9Bv')
target_gis = arcgis.gis.GIS(target_gis_url, target_gis_username, target_gis_password)
print("Source Environment: {} @ {}".format(src_gis._username, src_gis))
print("Target Environment: {} @ {}".format(target_gis._username, target_gis))

Source Environment: geodatanyeveier @ GIS @ https://nyeveier.maps.arcgis.com version:8.4
Target Environment: nvadmin @ GIS @ https://nyeveieras.maps.arcgis.com version:8.4


#### Validate how many items are in the group to be copied.

In [2]:
src_group = src_gis.groups.get(src_group_id)
src_group_content = src_group.content()
print("{} Total items in Group to be copied".format(len(src_group_content)))

3 Total items in Group to be copied


#### Execute the copy! This code below also accounts for hosted tile layers, navigating the item relationship back to the source item, copying that item, and republishing it in the target system.


In [3]:
for src_item in src_group_content:
    print("Copying Item: {} - {}".format(src_item.title, src_item.type))
    try:
        #Handle Hosted Tile Layers
        if 'Hosted Service' in src_item.typeKeywords and 'Map Service' in src_item.typeKeywords:
            if len(target_gis.content.search("source-{}".format(src_item.id)))>0:
                print("\t Item Already copied")
                continue
            #Clone TPK:
            print("\t Hosted Tile Layer found, copying source tile package")
            tpk_result = target_gis.content.clone_items(src_item.related_items('Service2Data'),folder=target_folder_name)
            if tpk_result == []:
                tpk_result = target_gis.content.search("source-{}".format(src_item.related_items('Service2Data')[0].id))
            print("\t Source Tile Package copied as {}".format(tpk_result[0].id))
            print("\t Publishing service...")
            service_result = tpk_result[0].publish(build_initial_cache=True)
            service_result.update({"typeKeywords":service_result.typeKeywords+["source-{}".format(src_item.id)]})
            print("\t Service Result: {}".format(service_result))
        else:
            result = target_gis.content.clone_items([src_item],folder=target_folder_name)
            if result == []:
                print("\t Item Already copied")
            else:
                print("\t Result: {}".format(result))
    except Exception as e:
        if "already exists" in str(e):
            print("\tService already copied to target organization")
        else:
            print("\t Error copying item: {} -- {} ".format(src_item.title, e))

Copying Item: 401 YM editering - Web Map
	 Result: [<Item title:"Båndlegginssoner" type:Feature Layer Collection owner:nvadmin>, <Item title:"401_YM_edit_P_YMLandgangenRugtvedt" type:Feature Layer Collection owner:nvadmin>, <Item title:"401_YM_edit_L_YMLandgangenRugtvedt" type:Feature Layer Collection owner:nvadmin>, <Item title:"401 YM editering" type:Web Map owner:nvadmin>]
Copying Item: 401_YM_edit_L_YMLandgangenRugtvedt - Feature Service
	 Item Already copied
Copying Item: 401_YM_edit_P_YMLandgangenRugtvedt - Feature Service
	 Item Already copied
